In [ ]:
import os
import sys

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from research_notebooks.grid_strike.grid_strike_bt import GridStrikeBT
from core.features.candles.volume import VolumeConfig
from core.features.candles.volatility import VolatilityConfig
from research_notebooks.grid_strike.utils import generate_report
import pandas as pd
import datetime
from controllers.generic.grid_strike import GridStrikeConfig
from core.backtesting.optimizer import BacktestingConfig, BaseStrategyConfigGenerator
from decimal import Decimal
from core.data_sources.clob import CLOBDataSource



class GridStrikeConfigGenerator(BaseStrategyConfigGenerator):
    def __init__(self, start_date: datetime.datetime, end_date: datetime.datetime, backtester= GridStrikeBT()):
        super().__init__(start_date, end_date, backtester)
        self.report = None
        self.trading_pairs = None
        self.candles = None

    async def generate_top_markets_report(self,
                                          connector_name: str,
                                          interval: str,
                                          days: int,
                                          quote_asset: str,
                                          min_notional_size: float,
                                          volatility_window: int,
                                          batch_size: int,
                                          sleep_time: float,
                                          volatility_threshold: float,
                                          volume_threshold: float,
                                          ) -> pd.DataFrame:
        clob = CLOBDataSource()
        trading_rules = await clob.get_trading_rules(connector_name)
        trading_pairs = trading_rules.filter_by_quote_asset(quote_asset).filter_by_min_notional_size(Decimal(min_notional_size)).get_all_trading_pairs()
        candles = await clob.get_candles_batch_last_days(connector_name, trading_pairs, interval, days, batch_size, sleep_time)
        report_df = generate_report(candles, volatility_config=VolatilityConfig(window=volatility_window), volume_config=VolumeConfig(short_window=5, long_window=20))
        natr_percentile = report_df['mean_natr'].quantile(volatility_threshold)
        volume_percentile = report_df['average_volume_per_hour'].quantile(volume_threshold)

        filtered_df = report_df[
            (report_df['mean_natr'] > natr_percentile) &
            (report_df['average_volume_per_hour'] > volume_percentile)
            ]
        self.report = filtered_df.sort_values(by='normalized_score', ascending=False)
        self.trading_pairs = self.report['trading_pair'].to_list()
        self.candles = {candle.trading_pair: candle for candle in candles}

    def generate_config(self, trial) -> BacktestingConfig:
        # Suggest hyperparameters using the trial object
        connector_name = "binance"
        total_amount_quote = 1000
        trading_pair = trial.suggest_categorical("trading_pair", self.trading_pairs)
        grid_mid_price = self.report[self.report['trading_pair'] == trading_pair]['grid_mid_price'].values[0]
        sniper_upper_price = self.report[self.report['trading_pair'] == trading_pair]['sniper_upper_price'].values[0]
        sniper_lower_price = self.report[self.report['trading_pair'] == trading_pair]['sniper_lower_price'].values[0]
        grid_step = trial.suggest_float("grid_step", 0.005, 0.02, step=0.005)
        grid_upper_price_quantile = trial.suggest_float("grid_upper_price_quantile", 0.75, 0.95, step=0.05)
        grid_lower_price_quantile = trial.suggest_float("grid_lower_price_quantile", 0.05, 0.25, step=0.05)
        candles = self.candles[trading_pair]
        grid_upper_price = candles.data.close.quantile(grid_upper_price_quantile)
        grid_lower_price = candles.data.close.quantile(grid_lower_price_quantile)

        # Create the strategy configuration
        config = GridStrikeConfig(
            connector_name=connector_name,
            trading_pair=trading_pair,
            total_amount_quote=Decimal(total_amount_quote),
            grid_upper_price=grid_upper_price,
            grid_mid_price=grid_mid_price,
            grid_lower_price=grid_lower_price,
            sniper_upper_price=sniper_upper_price,
            sniper_lower_price=sniper_lower_price,
            grid_step=grid_step,
            rebalanced=True,
        )

        # Return the configuration encapsulated in BacktestingConfig
        return BacktestingConfig(config=config, start=self.start, end=self.end)

In [ ]:
from core.backtesting.optimizer import StrategyOptimizer
import datetime

connector_name = "binance"
interval = "15m"
days = 30
quote_asset = "USDT"
min_notional_size = 1
volatility_window = 20
batch_size = 100
sleep_time = 2.0
volatility_threshold = 0.4
volume_threshold = 0.4

start_date = datetime.datetime(2024, 9, 10)
end_date = datetime.datetime(2024, 9, 12)
config_generator = GridStrikeConfigGenerator(start_date=start_date, end_date=end_date)
await config_generator.generate_top_markets_report(
    connector_name=connector_name, interval=interval, days=days, quote_asset=quote_asset,
    min_notional_size=min_notional_size, volatility_window=volatility_window, batch_size=batch_size, sleep_time=sleep_time, volatility_threshold=volatility_threshold, volume_threshold=volume_threshold)


In [ ]:
optimizer = StrategyOptimizer(root_path=root_path)
await optimizer.optimize(
    study_name="grid_strike_2",
    config_generator=config_generator,
    n_trials=1000,
)

In [ ]:
optimizer.launch_optuna_dashboard()